In [ ]:
! wget https://huggingface.co/datasets/ntt123/viet-tts-dataset/resolve/main/viet-tts.tar.gz -O viet-tts.tar.gz 

--2024-04-23 09:44:21--  https://huggingface.co/datasets/ntt123/viet-tts-dataset/resolve/main/viet-tts.tar.gz
Resolving huggingface.co (huggingface.co)... 18.67.216.51, 18.67.216.50, 18.67.216.43, ...
Connecting to huggingface.co (huggingface.co)|18.67.216.51|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/b6/a0/b6a06f111e29ec13cce1bd8ac23c03399a91a255436d45e3107e1166e1aae0eb/e00f71ccfe907cd74a9cb1bcb60d058ec8a9aa8c4c101d019dbeeefc588c5d0b?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27viet-tts.tar.gz%3B+filename%3D%22viet-tts.tar.gz%22%3B&response-content-type=application%2Fgzip&Expires=1714099462&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNDA5OTQ2Mn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9iNi9hMC9iNmEwNmYxMTFlMjllYzEzY2NlMWJkOGFjMjNjMDMzOTlhOTFhMjU1NDM2ZDQ1ZTMxMDdlMTE2NmUxYWFlMGViL2UwMGY3MWNjZmU5MDdjZDc0YTljYjFiY2I2MGQwNThl

In [ ]:
! mkdir -p dataset

In [ ]:
! tar -C dataset -xzf viet-tts.tar.gz

In [ ]:
import csv

input_file = 'dataset/meta_data.tsv'
output_file = 'dataset/transcript.txt'

with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', encoding='utf-8') as outfile:
    # Tạo một đối tượng reader để đọc tsv
    tsv_reader = csv.reader(infile, delimiter='\t')

    for row in tsv_reader:
        if row:  # Kiểm tra xem hàng không rỗng
            wav_filename = row[0].split('/')[-1]
            transcript = row[1]
            
            # Ghi vào tệp đầu ra với định dạng mong muốn
            outfile.write(f"{wav_filename}|{transcript}\n")

In [1]:
! pip install TTS

  Obtaining dependency information for TTS from https://files.pythonhosted.org/packages/6e/36/dce195a2b9ff3a11863f53fc35109f8e1f2c7d40419f999f9d0a12cc23e5/TTS-0.22.0-cp310-cp310-manylinux1_x86_64.whl.metadata
  Obtaining dependency information for scipy>=1.11.2 from https://files.pythonhosted.org/packages/b9/9d/39dbcf49a793157f9d4f5b8961855677eb4dbb4b82700dcee7042ad2310c/scipy-1.13.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 703.3 kB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for torch>=2.1 from https://files.pythonhosted.org/packages/33/b3/1fcc3bccfddadfd6845dcbfe26eb4b099f1dfea5aa0e5cfb92b3c98dba5b/torch-2.2.2-cp310-cp310-manylinux1_x86_64.whl.metadata
  Obtaining dependency information for soundfile>=0.12.0 from https://files.pythonhosted.org/packages/c1/07/7591f4efd29e65071c3a61b53725036ea8f73366a4920a481ebddaf8d0ca/soundfile-0.12.1-py2.py3-none-manylinux_2_31_x86_64.whl.metadata
  Ob

In [1]:
import os

from trainer import Trainer, TrainerArgs

from TTS.tts.configs.shared_configs import BaseDatasetConfig, CharactersConfig
from TTS.tts.configs.vits_config import VitsConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.models.vits import Vits, VitsArgs, VitsAudioConfig
from TTS.tts.utils.speakers import SpeakerManager
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor

In [2]:
output_path = "/home/maicg/Documents/Me/python-image-processing/tts/coqui_tts"


dataset_config = BaseDatasetConfig(
    formatter="ljspeech", meta_file_train="transcript.txt", path=os.path.join(output_path, "dataset/")
)

In [3]:
audio_config = VitsAudioConfig(
    sample_rate=44100, win_length=1024, hop_length=256, num_mels=80, mel_fmin=0, mel_fmax=None
)

In [4]:
character_config = CharactersConfig(
    characters_class= "TTS.tts.models.vits.VitsCharacters",
    characters= "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz1234567890đáàảãạăắặằẳẵâấầẩẫậéèẻẽẹêếềểễệíìỉĩịóòỏõọôốồổỗộơớờởỡợúùủũụưứừửữựýỳỷỹỵ",
    punctuations=" !,.?-:",
    pad= "<PAD>",
    eos= "<EOS>",
    bos= "<BOS>",
    blank= "<BLNK>",
)

In [5]:
config = VitsConfig(
    audio=audio_config,
    characters=character_config,
    run_name="vits_vctk",
    batch_size=16,
    eval_batch_size=4,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=0,
    epochs=1000,
    text_cleaner="basic_cleaners",
    use_phonemes=False,
    phoneme_language="en-us",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    compute_input_seq_cache=True,
    print_step=25,
    print_eval=False,
    save_best_after=1000,
    save_checkpoints=True,
    save_all_best=True,
    mixed_precision=True,
    max_text_len=250,  # change this if you have a larger VRAM than 16GB
    output_path=output_path,
    datasets=[dataset_config],
    cudnn_benchmark=False,
    test_sentences=[
        ["Xin chào"],
        ["Bạn có khỏe không?"],
        ["Tôi có thể giúp gì cho bạn?"]
    ]
)

In [6]:
# Audio processor is used for feature extraction and audio I/O.
# It mainly serves to the dataloader and the training loggers.
ap = AudioProcessor.init_from_config(config)

# INITIALIZE THE TOKENIZER
# Tokenizer is used to convert text to sequences of token IDs.
# config is updated with the default characters if not defined in the config.
tokenizer, config = TTSTokenizer.init_from_config(config)

 > Setting up Audio Processor...
 | > sample_rate:44100
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


In [7]:
def formatter(root_path, manifest_file, **kwargs):  # pylint: disable=unused-argument
    """Assumes each line as ```<filename>|<transcription>```
    """
    txt_file = os.path.join(root_path, manifest_file)
    items = []
    speaker_name = "my_speaker"
    with open(txt_file, "r", encoding="utf-8") as ttf:
        for line in ttf:
            cols = line.split("|")
            wav_file = f"/home/maicg/Documents/Me/python-image-processing/tts/coqui_tts/dataset/wavs/{cols[0]}"
            text = cols[1]
            # print(text)
            items.append({"text":text, "audio_file":wav_file, "speaker_name":speaker_name, "root_path": root_path})
    return items


In [8]:
train_samples, eval_samples = load_tts_samples(
dataset_config, 
eval_split=True, 
formatter=formatter)

 | > Found 22884 files in /home/maicg/Documents/Me/python-image-processing/tts/coqui_tts/dataset


In [9]:
# init model
model = Vits(config, ap, tokenizer, speaker_manager=None)

# init the trainer and 🚀
trainer = Trainer(
    TrainerArgs(),
    config,
    output_path,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples,
)

/home/maicg/anaconda3/envs/deepfake/lib/python3.10/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11080). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: True
 | > Precision: fp16
 | > Num. of CPUs: 12
 | > Num. of Torch Threads: 6
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False
2024-04-23 15:26:29.577431: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results

RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xf

SystemError: <built-in function __import__> returned a result with an exception set


 > Model has 83060524 parameters


In [10]:
trainer.fit()


 > EPOCH: 0/1000
 --> /home/maicg/Documents/Me/python-image-processing/tts/coqui_tts/vits_vctk-April-23-2024_03+26PM-813ae8f

 > TRAINING (2024-04-23 15:26:35) 




> DataLoader initialization
| > Tokenizer:
	| > add_blank: True
	| > use_eos_bos: False
	| > use_phonemes: False
| > Number of instances : 22656
 | > Preprocessing samples
 | > Max text length: 250
 | > Min text length: 4
 | > Avg text length: 95.75106477373558
 | 
 | > Max audio length: 641053.0
 | > Min audio length: 20509.0
 | > Avg audio length: 247491.00248447206
 | > Num. instances discarded samples: 116
 | > Batch group size: 0.


/home/maicg/anaconda3/envs/deepfake/lib/python3.10/site-packages/torch/functional.py:660: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at ../aten/src/ATen/native/SpectralOps.cpp:874.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]
